In [2]:
from visiprog.data import *
from visiprog.metriclearning import *
from visiprog.evaluate import *
%matplotlib inline
import numpy as np
from visiprog.metric_learn.lfda_visiprog import LFDA_VISIPROG
from metric_learn import Covariance

from IPython.display import Image, display
from sklearn.model_selection import train_test_split
import pandas as pd

import random

from sklearn.cluster import KMeans

In [3]:
raw_feature = np.genfromtxt('visiprog/data/curetaliasfree.csv',delimiter=",")
label_material = read_material_label()
groups_VSP, material = read_VSP_group(pappas_only=False)

D = count_matrix(groups_VSP, raw_feature.shape[0])
clusters_VSP = spectralClusteringDBSCAN(D, threshold=0,eps=0.001)

print(raw_feature.shape)
print(label_material.shape)

(5245, 82)
(5245,)


In [4]:
label_visiprog = get_VSP_label(groups_VSP, raw_feature.shape[0])
label_VSP_clusters = get_VSP_label(clusters_VSP, raw_feature.shape[0])

In [5]:
model = Covariance()
model.fit(raw_feature)
X_STSIMM = model.transform(raw_feature)

In [6]:
accuracy_STSIMM = []
accuracy_mat = []
accuracy_vsp_original = []
accuracy_vsp = []

n_cluster = 1

N_material = 61
N_material_train = 50

material_list = list(range(61))

for fold in range(10):
    print("Trial {}".format(fold))

    # randomly select N_material_train material for training the metric
    # and test on the rest
    train_select = np.zeros_like(label_material).astype(bool)

    random.shuffle(material_list)

    idx_train = np.array([])
    for train_class in material_list[:N_material_train]:
        train_select |= (label_material == train_class)
            
    idx_train = np.where(train_select == True)[0]
    idx_test = np.where(train_select == False)[0]

    idx_train = idx_train.astype(int)
    idx_test = idx_test.astype(int)

    print("Training shape {} and test shape {}".format(len(idx_train), len(idx_test)))

    X_train_metric = raw_feature[idx_train,:]
    X_STSIMM_train_metric = X_STSIMM[idx_train,:]
    Y_VSP_train_metric = label_visiprog[idx_train]
    Y_mat_train_metric = label_material[idx_train]

    # now split the remaining data into train and test
    X_train, X_test, X_STSIMM_train, X_STSIMM_test, Y_mat_train, Y_mat_test, Y_VSP_train, Y_VSP_test = \
        train_test_split(raw_feature[idx_test,:], X_STSIMM[idx_test,:], \
                         label_material[idx_test], label_visiprog[idx_test], \
                         test_size=0.2, stratify=label_material[idx_test])
    
    res_STSIMM = kmean_subclass(X_STSIMM_train, Y_mat_train, X_STSIMM_test, Y_mat_test, n_cluster)
    accuracy_STSIMM.append(res_STSIMM['accuracy'])

    model = LFDA()
    try:
        model.fit(X_train_metric, Y_mat_train_metric)
        X_mat_train = model.transform(X_train)
        X_mat_test = model.transform(X_test)
        res = kmean_subclass(X_mat_train, Y_mat_train, X_mat_test, Y_mat_test, n_cluster)
        accuracy_mat.append(res['accuracy'])
    except Exception as e:
        print(e)
        acuracy_mat.append(np.nan)

    model_vsp_original = LFDA()
    try:
        # remove the unlabeled data
        idx_outlier = np.where(Y_VSP_train == 0)[0]
        X_train_without_outlier = X_train_metric[~idx_outlier,:]
        Y_VSP_train_without_outlier = Y_VSP_train_metric[~idx_outlier]
        
        model_vsp_original.fit(X_train_without_outlier, Y_VSP_train_without_outlier)
        X_vsp_original_train = model_vsp_original.transform(X_train)
        X_vsp_original_test = model_vsp_original.transform(X_test)
        res_vsp_original = kmean_subclass(X_vsp_original_train, Y_mat_train, \
                                          X_vsp_original_test, Y_mat_test, n_cluster)
        accuracy_vsp_original.append(res_vsp_original['accuracy'])
    except Exception as e:
        print(e)
        accuracy_vsp_original.append(np.nan)

    model_vsp = LFDA_VISIPROG()
    try:
        model_vsp.fit(X_train_metric, Y_VSP_train_metric)
        X_vsp_train = model_vsp.transform(X_train)
        X_vsp_test = model_vsp.transform(X_test)
        res_vsp = kmean_subclass(X_vsp_train, Y_mat_train, X_vsp_test, Y_mat_test, n_cluster)
        accuracy_vsp.append(res_vsp['accuracy'])
    except Exception as e:
        print(e)
        accuracy_vsp.append(np.nan)


df = pd.DataFrame({'visiprog':accuracy_vsp, 'material':accuracy_mat, 'STSIMM':accuracy_STSIMM, \
       'visiprog_original':accuracy_vsp_original}, columns=['STSIMM','visiprog_original','visiprog','material'])

df.loc['average'] = df.mean()

display(df)
df.to_csv('thrasos/results/across_k{}.csv'.format(n_cluster))

Trial 0
Training shape 4146 and test shape 1099
Trial 1
Training shape 4253 and test shape 992
Trial 2
Training shape 4150 and test shape 1095
Trial 3
Training shape 4422 and test shape 823


/home/andrey/anaconda2/envs/py34/lib/python3.4/site-packages/metric_learn/lfda.py:112: RuntimeWarning: invalid value encountered in sqrt
  vecs *= np.sqrt(vals)


Input contains NaN, infinity or a value too large for dtype('float64').
Trial 4
Training shape 4257 and test shape 988
Trial 5
Training shape 4142 and test shape 1103
Trial 6
Training shape 4329 and test shape 916
Input contains NaN, infinity or a value too large for dtype('float64').
Trial 7
Training shape 4534 and test shape 711
the leading minor of order 68 of 'b' is not positive definite. The factorization of 'b' could not be completed and no eigenvalues or eigenvectors were computed.
Trial 8
Training shape 4325 and test shape 920
Trial 9
Training shape 4141 and test shape 1104


,STSIMM,visiprog_original,visiprog,material
0,0.904545,0.681818,0.786364,0.790909
1,0.939698,0.773869,0.874372,0.909548
2,0.913242,0.744292,0.785388,0.812785
3,0.969697,NaN,0.842424,0.945455
4,0.914141,0.585859,0.843434,0.883838
5,0.891403,0.742081,0.914027,0.895928
6,0.918478,NaN,0.875000,0.902174
7,0.937063,NaN,0.902098,0.867133
8,0.907609,0.630435,0.885870,0.831522
9,0.918552,0.823529,0.936652,0.895928
